# Distances

In [1]:
from koselleck import *

## Generating data

In [2]:
FORCE=False
YMIN=1720
YMAX=1960

### Within-model Distance matrices

In [3]:
# gen_within_model_dists(
#     words=get_valid_words(),
#     num_proc=1,
#     period_len=5,
#     ymin=YMIN,
#     ymax=YMAX,
#     force=FORCE
# )

### Across-model Distance matrices

In [4]:
# dfpaths_cmp=get_cross_model_dists_paths(
#     period_len=5,
#     ymin=YMIN,
#     ymax=YMAX,
#     force=FORCE,
# )
# # dfpaths_cmp

In [5]:
# odf=gen_cross_model_dists(
#     dfpaths=dfpaths_cmp,
#     lim=None,
#     num_proc=4,
#     num_runs=4,
#     force=1#FORCE
# )
# stop

In [6]:
# odf.query('word=="station"').groupby(['period1','period2']).mean().sort_index()

## Postprocessing

In [9]:
# This averages down
alldist_df = get_cross_model_dists(force=FORCE)
alldist_df

dist_local  dist_local_z  \
corpus1 corpus2 period1   period2   word                               
bpo     bpo     1890-1895 1895-1900 day       0.007251     -2.229570   
                1865-1870 1870-1875 time      0.007274     -2.232159   
                1930-1935 1925-1930 day       0.007437     -2.227280   
                1865-1870 1875-1880 time      0.007450     -2.230666   
                1880-1885 1875-1880 time      0.007642     -2.226431   
...                                                ...           ...   
                1810-1815 1925-1930 factor    0.578083      8.208947   
                1805-1810 1930-1935 factor    0.578762      8.230830   
                1810-1815 1910-1915 factor    0.579372      8.227624   
                          1895-1900 factor    0.580439      8.245224   
                1930-1935 1810-1815 factor    0.591988      8.469728   

                                            dist_local_perc  
corpus1 corpus2 period1   period2   word                     
bpo     bpo     1890-1895 1895-1900 day            0.003468  
                1865-1870 1870-1875 time           0.003162  
                1930-1935 1925-1930 day            0.006026  
                1865-1870 1875-1880 time           0.007257  
                1880-1885 1875-1880 time           0.006189  
...                                                     ...  
                1810-1815 1925-1930 factor        99.999679  
                1805-1810 1930-1935 factor        99.999763  
                1810-1815 1910-1915 factor        99.999791  
                          1895-1900 factor        99.999755  
                1930-1935 1810-1815 factor        99.999883  

[5515535 rows x 3 columns]

In [14]:
dfm_word = alldist_df.groupby('word').mean().sort_values('dist_local_perc').tail(25)
# dfm_word

## Plotting data

In [ ]:
# get_all_localdists()

In [ ]:
# source(get_historical_semantic_distance_matrix)

In [12]:
# odf=get_historical_semantic_distance_matrix('station')
# # odf[(odf.period1=='1720-1725') | (odf.period2=='1720-1725')]
# odf

In [15]:
# @interact
# def doplot(w='station'):
#     return plot_historical_semantic_distance_matrix(w)

In [ ]:
plot_historical_semantic_distance_matrix('station')

In [ ]:
plot_historical_semantic_distance_matrix('interest')

In [ ]:
plot_historical_semantic_distance_matrix('commerce')